In [1]:
import powerplantmatching as pm
import linopy
import numpy as np
import pandas as pd
import geopandas as gpd
import os

ERROR 1: PROJ: proj_create_from_database: Open of /trinity/home/thomgeo/micromamba/envs/open_eraa2/share/proj failed


In [2]:
def prepare_power_plant_table():

    res = ['Wind', 'Solar', "Hydro"]
    
    match_carriers = {
        'Nuclear': "nuclear",
        'Hard Coal': "coal",
        'Lignite': "lignite",
        'Natural Gas': np.nan,
        'Oil': "oil",
        'Solid Biomass': "biomass",
        'Other': "other",
        'Biogas': "biomass",
        'Waste': "biomass",
        'Geothermal': "geothermal"
    }
    
    power_plants = pm.powerplants()
    power_plants = power_plants.query("(Fueltype not in @res)")
    power_plants = power_plants[~(power_plants.DateOut < 2005)]
    
    power_plant_locations = gpd.GeoDataFrame(index=power_plants.index, geometry=gpd.points_from_xy(power_plants.lon, power_plants.lat), crs="EPSG:4326")
    power_plant_locations = power_plant_locations.to_crs("EPSG:3035")
    
    plant_to_zone = gpd.sjoin_nearest(
        power_plant_locations,
        bidding_zones,
        how="left"
    )
    
    plant_to_zone = plant_to_zone.groupby(plant_to_zone.index).last()
    power_plants["bus"] = power_plants.index.map(plant_to_zone.zone)

    power_plants["carrier"] = power_plants.Fueltype.map(match_carriers)

    for plant in power_plants.index[power_plants.carrier.isna()]:
        if power_plants.loc[plant, "Technology"] == "CCGT":
            power_plants.loc[plant, "carrier"] = "CCGT"
        else:
            power_plants.loc[plant, "carrier"] = "OCGT"
    
    power_plants.loc[(power_plants.bus=="ES00") & (power_plants.Set == "CHP"), "carrier"] = "other"
    power_plants = power_plants[power_plants.Capacity>0]
    
    return power_plants

In [3]:
def get_age_matching():
    
    age_matching = pd.DataFrame(index=technology_parameters.index, dtype=str)

    age_matching["from"] = np.nan
    age_matching["to"] = np.nan
    
    idx = pd.IndexSlice
    
    age_matching.loc[idx[:, "old 1"], "from"] = 1900
    age_matching.loc[idx[:, "old 1"], "to"] = 1980
    age_matching.loc[idx[:, "old 2"], "from"] = 1980
    age_matching.loc[idx[["coal", "lignite", "oil", "OCGT"], "old 2"], "to"] = 2010
    age_matching.loc[idx[["CCGT"], "old 2"], "to"] = 2000
    age_matching.loc[idx[["coal", "lignite", "oil", "OCGT"], "new"], "from"] = 2010
    age_matching.loc[idx[:, "new"], "to"] = 2040
    age_matching.loc[idx[:, "present 1"], "from"] = 2000
    age_matching.loc[idx[:, "present 1"], "to"] = 2010
    age_matching.loc[idx[:, "present 2"], "from"] = 2010
    age_matching.loc[idx[:, "present 2"], "to"] = 2020
    age_matching.loc[idx[["CCGT"], "new"], "from"] = 2020
    age_matching.drop(["old"], level=1, inplace=True)
    age_matching.drop("CCS", level=1, inplace=True)
    age_matching.drop(('oil','-'), inplace=True)
    age_matching.loc[["nuclear", "hydrogen", "biomass", "other"], :] = 1900, 2040

    return age_matching

In [4]:
def add_technical_technology_parameters_existing(all_plants, detailed_plants, technology_parameters, age_matching):

    idx = pd.IndexSlice
    
    all_plants["age_class"] = np.nan
    
    for plant in detailed_plants.index:
    
        datein = power_plants.loc[plant, "DateIn"]
        carrier = power_plants.loc[plant, "carrier"]
        
        if (carrier in age_matching.index.levels[0]) and not np.isnan(datein):
            all_plants.loc[plant, "age_class"] = age_matching[(age_matching["from"] <= datein)&(age_matching["to"] > datein)].loc[carrier].index[0]
        elif (carrier in age_matching.index.levels[0]) and ("old 2" in age_matching.loc[carrier].index):
            all_plants.loc[plant, "age_class"] = "old 2"    
    
    all_plants.age_class = all_plants.age_class.fillna("new")
    all_plants.loc[all_plants.carrier.isin(["biomass", "nuclear", "other"]), "age_class"] = "-"
    all_plants.loc[all_plants.carrier =="hydrogen", "age_class"] = "CCGT"
    
    all_plants['invest_status'] = "existing"
    all_plants['p_nom_max'] = all_plants.p_nom
    all_plants['p_nom_min'] = 0.01
    
    all_plants[['start_up_cost', 'ramp_limit_up', 'ramp_limit_down', 'p_min_pu', 'min_up_time', 'min_down_time', 'var_om' ]] = np.nan
    
    start_up_cost = []
    ramp_limit_up = []
    ramp_limit_down = []
    p_min_pu = []
    min_up_time = []
    min_down_time = [] 
    var_om = [] 
    efficiency = []
    
    for plant in all_plants.index:
    
        carrier = all_plants.loc[plant, "carrier"]
        age_class = all_plants.loc[plant, "age_class"]
        
        start_up_cost.append(technology_parameters.loc[idx[carrier, age_class], "start_up_fix_cost"])
        ramp_limit_up.append(technology_parameters.loc[idx[carrier, age_class], "ramp_limit_up"])
        ramp_limit_down.append(technology_parameters.loc[idx[carrier, age_class], "ramp_limit_down"])
        p_min_pu.append(technology_parameters.loc[idx[carrier, age_class], "p_min_pu"])
        min_up_time.append(technology_parameters.loc[idx[carrier, age_class], "min_up_time"])
        min_down_time.append(technology_parameters.loc[idx[carrier, age_class], "min_down_time"])
        var_om.append(technology_parameters.loc[idx[carrier, age_class], "var_OM"])
        efficiency.append(technology_parameters.loc[idx[carrier, age_class], "efficiency"])
    
    all_plants.start_up_cost = start_up_cost
    
    """
    all_plants.ramp_limit_down_real = ramp_limit_down
    all_plants.ramp_limit_down = all_plants.ramp_limit_down_real.clip(upper=1.) 
    all_plants.ramp_limit_up_real = ramp_limit_up
    all_plants.ramp_limit_up = all_plants.ramp_limit_up_real.clip(upper=1.)
    """
    
    all_plants.ramp_limit_down = ramp_limit_down
    all_plants.ramp_limit_up = ramp_limit_up
    all_plants.p_min_pu = p_min_pu
    all_plants.min_up_time = min_up_time
    all_plants.min_down_time = min_down_time
    all_plants.var_om = var_om
    all_plants.efficiency = efficiency

    all_plants.p_nom = all_plants.p_nom.astype(float)
    all_plants.p_nom_max = all_plants.p_nom_max.astype(float)

    return all_plants


In [5]:
def aggregate_small(eraa_plants_detailed):

    small_plants = eraa_plants_detailed[eraa_plants_detailed.p_nom.round()<=de_minimis]
    
    agg_func = pd.Series(
        "mean",
        index = ['p_nom', 'efficiency',  'p_nom_max', 'p_nom_min', 'start_up_cost', 'ramp_limit_up',
                 'ramp_limit_down', 'p_min_pu', 'min_up_time', 'min_down_time', 'var_om', "age_class"]
    )
    
    agg_func.loc[["p_nom", "p_nom_max"]] = "sum"
    agg_func.loc["age_class"] = pd.Series.mode
    
    aggregated_small = small_plants.groupby(["bus", "carrier", "entry", "exit", 'invest_status', "cluster"]).agg(agg_func).reset_index()
    small_index = pd.Index(aggregated_small.bus + " small " + aggregated_small.carrier + " ")
    aggregated_small.index = small_index + pd.Series(small_index).to_frame().groupby(0).cumcount().astype(str)
    
    eraa_plants = eraa_plants_detailed.drop(small_plants.index)
    return pd.concat([eraa_plants, aggregated_small])

In [6]:
def build_new_investments(df, properties):

    technologies_for_investment = ["OCGT", "CCGT"]
    years = df.entry.unique()
    zones_for_investment = df.bus.unique()

    new = pd.DataFrame(
        0.01, 
        index= pd.MultiIndex.from_product(
            [zones_for_investment,technologies_for_investment, years], 
            names=["bus", "carrier", "entry"]), 
        columns=["p_nom"]
    )

    new.reset_index(inplace=True)
    new.index = new.bus + " " + new.carrier + " new " + new.entry.astype(str)
    new["efficiency"] = properties.loc[:, "new", :]["efficiency"].reindex(new.carrier).values
    new["start_up_cost"] = properties["start_up_fix_cost"].loc[:, "new"].reindex(new.carrier, level=1).fillna(0).values
    new["ramp_limit_up"] = properties["ramp_limit_up"].loc[:, "new"].reindex(new.carrier).values*60
    new["ramp_limit_down"] = properties["ramp_limit_down"].loc[:, "new"].reindex(new.carrier).values*60
    new["p_min_pu"] = properties["p_min_pu"].loc[:, "new"].reindex(new.carrier).values
    new["min_up_time"] = properties["min_up_time"].loc[:, "new"].reindex(new.carrier).values
    new["min_down_time"] = properties["min_down_time"].loc[:, "new"].reindex(new.carrier).values
    new["exit"] = target_years[-1] + 1
    new["p_nom_max"] = np.inf
    new["p_nom_min"] = 0.01
    new["var_om"] = properties.loc[:, "new",:].reindex(new.carrier)["var_OM"].values
    new["invest_status"] = "new"
    new["age_class"] = "new"
    
    return pd.concat([df, new])

In [7]:
def match_to_cluster(bus, carrier):
    
    capacity = power_plants.query("(carrier == @carrier) and (bus == @bus)").Capacity.copy()
    clusters = aggregated_plants.query("bus == @bus and carrier == @carrier").copy()
    clusters = clusters.filter(like="exit", axis=0)
    
    if (len(capacity)>0) and (len(clusters)>0):
        
        m = linopy.Model()    
        clusters.index.name = "cluster"      
        cluster_matching = m.add_variables(binary=True, coords=[clusters.index, capacity.index], name="cluster_matching")       
        matched_capacity = m.add_variables(lower=0, coords=[clusters.index], name="matched_capacity")
        
        m.add_constraints(
            matched_capacity <= clusters.p_nom,
            name="upper_limit_is_total"
        )
                
        m.add_constraints(
            (cluster_matching*capacity).sum("id") == matched_capacity,
            name="match_plants_to_cluster"
        )
        
        m.add_constraints(
            cluster_matching.sum("cluster") <= 1,
            name="unique_assignment"
        )
        
        m.add_objective((matched_capacity**2).sum() - (2*clusters.p_nom*matched_capacity).sum())       
        m.solve(solver_name="cplex", **solver_options)   
        
        return m.solution["cluster_matching"].to_series(), m.termination_condition
    
    else:
        return pd.Series([]), "optimal"

In [8]:
def match_existing_plants():
    
    matched_plants=[]
    status = pd.Series(index= pd.unique(pd.MultiIndex.from_frame(aggregated_plants[["bus", "carrier"]])), dtype=str)
    for i in status.index:
        bus, carrier = i
        print(bus, carrier)
        matched, status[i] = match_to_cluster(bus, carrier)
        matched_plants.append(matched)
    
    if (status != "optimal").sum() == 0:
        print("Large decomposition finished successfully")
    else:
        raise ValueError('Not all decompositions converged')
    
    matched_plants = pd.concat(matched_plants)
    matched_plants.index = pd.MultiIndex.from_tuples(matched_plants.index)
    matched_plants = matched_plants.round(0).astype(int)
    
    cluster_association = matched_plants[matched_plants == 1].reset_index(0).level_0
    cluster_association.name="cluster"
    
    detailed_plants = power_plants[["bus", "carrier", "Capacity", "Efficiency"]].copy()
    detailed_plants.columns = detailed_plants.columns.str.replace("Efficiency", "efficiency").str.replace("Capacity", "p_nom")
    detailed_plants = detailed_plants.reindex(cluster_association.index)
    detailed_plants["cluster"] = cluster_association
    detailed_plants.sort_index(inplace=True)
    detailed_plants["invest_status"] = "existing"
    detailed_plants["entry"] = target_years[0]
    detailed_plants["exit"] = [int(i[1]) for i in cluster_association.str.split("exit ")]
    
    return detailed_plants, matched_plants

In [9]:
def build_generic_plants():
    
    unmatched_capacity = aggregated_plants.query("invest_status == 'existing'").p_nom.subtract(
        matched_plants.unstack(0).multiply(power_plants.loc[matched_plants.index.levels[1], "Capacity"], axis=0).sum(),
        fill_value=0
    )
    
    typical_size_unmatched = typical_size.reindex(aggregated_plants.reindex(unmatched_capacity.index).carrier.values)
    typical_size_unmatched.index = unmatched_capacity.index
    n_generic = (unmatched_capacity//typical_size_unmatched).astype(int)
    residual = unmatched_capacity.subtract(n_generic.multiply(typical_size_unmatched))
    
    generic_plants = []
    
    for cluster in n_generic[(n_generic>0)|(residual>0)].index:    
        generic_unit = aggregated_plants.loc[cluster][["bus", "carrier", "efficiency","p_nom", "entry", "exit", "invest_status"]].copy()
        generic_unit.p_nom = float(typical_size[generic_unit.carrier])
        generic_unit["cluster"] = cluster
        generic_unit["efficiency"] = np.nan
    
        add_on = " ".join(cluster.split(" ")[-2:])
        for number in range(n_generic[cluster]):
            unit_to_add = generic_unit.copy()
            unit_to_add.name = "{bus} {carrier} {add_on} {number}".format(bus=generic_unit["bus"], carrier=generic_unit["carrier"], number=number, add_on=add_on)
            generic_plants.append(unit_to_add)
    
        residual_unit = generic_unit.copy()
        residual_unit.name = "{bus} {carrier} {add_on} {number}".format(bus=generic_unit["bus"], carrier=generic_unit["carrier"], number=n_generic[cluster], add_on=add_on)
        residual_unit.p_nom = residual[cluster]
        generic_plants.append(residual_unit)
    
    generic_plants = pd.concat(generic_plants, axis=1).T

    return generic_plants

In [10]:
typical_size = pd.Series({
    "CCGT" : 400,
    "OCGT" : 50,
    "biomass" : 50,
    "coal" : 400,
    "lignite" : 400,
    "nuclear" : 800,
    "oil" : 50,
    "other": 50,
    "hydrogen":400
})

In [11]:
solver_name="cplex"

In [12]:
solver_options = {"mip.tolerances.mipgap" : 0.01}

In [13]:
solver_options = {
    "mip.tolerances.mipgap": 0.01,
    "emphasis.mip": 4,
    "threads": 4, 
    "lpmethod": 4, # barrier    "solutiontype": 2, # non basic solution, ie no crossover
    "barrier.convergetol": 1.e-2,
    "feasopt.tolerance": 1.e-6,
    "emphasis.numerical": 1,
}

In [14]:
aggregated_plants = pd.read_hdf("resources/dispatchable_capacities.h5")

In [15]:
technology_parameters = pd.read_hdf("resources/technology_parameters.h5")

In [16]:
bidding_zones = gpd.read_file("resources/bidding_zones.geojson")
bidding_zones = bidding_zones.to_crs("EPSG:3035")

/trinity/home/thomgeo/micromamba/envs/open_eraa2/lib/python3.12/site-packages/pyogrio/__init__.py:7: DeprecationWarning: The 'shapely.geos' module is deprecated, and will be removed in a future version. All attributes of 'shapely.geos' are available directly from the top-level 'shapely' namespace (since shapely 2.0.0).
  import shapely.geos  # noqa: F401


In [17]:
de_minimis = 50

In [18]:
power_plants = prepare_power_plant_table()

In [19]:
target_years = sorted(aggregated_plants.entry.unique())

In [20]:
detailed_plants, matched_plants = match_existing_plants()
generic_plants = build_generic_plants()
all_plants = pd.concat([detailed_plants, generic_plants])

AL00 CCGT
AT00 CCGT
Version identifier: 22.1.2.0 | 2024-12-10 | f4cec290b
CPXPARAM_Read_DataCheck                          1
CPXPARAM_LPMethod                                4
CPXPARAM_Threads                                 4
CPXPARAM_Emphasis_Numerical                      1
CPXPARAM_Emphasis_MIP                            4
CPXPARAM_MIP_Tolerances_MIPGap                   0.01
CPXPARAM_Barrier_ConvergeTol                     0.01
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIQP Presolve eliminated 4 rows and 7 columns.
Reduced MIQP has 17 rows, 49 columns, and 94 nonzeros.
Reduced MIQP has 45 binaries, 0 generals, 0 SOSs, and 0 indicators.
Reduced MIQP objective Q matrix has 4 nonzeros.
Presolve time = 0.00 sec. (0.08 ticks)
Probing time = 0.00 sec. (0.02 ticks)
Tried aggregator 1 time.
Reduced MIQP has 17 rows, 49 columns, and 94 nonzeros.
Reduced MIQP has 45 binaries, 1 generals, 0 SOSs, and 0 indicators.
Reduced MIQP objective Q matrix 

Dual values of MILP couldn't be parsed


BE00 CCGT
Version identifier: 22.1.2.0 | 2024-12-10 | f4cec290b
CPXPARAM_Read_DataCheck                          1
CPXPARAM_LPMethod                                4
CPXPARAM_Threads                                 4
CPXPARAM_Emphasis_Numerical                      1
CPXPARAM_Emphasis_MIP                            4
CPXPARAM_MIP_Tolerances_MIPGap                   0.01
CPXPARAM_Barrier_ConvergeTol                     0.01
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIQP Presolve eliminated 3 rows and 10 columns.
Reduced MIQP has 20 rows, 44 columns, and 85 nonzeros.
Reduced MIQP has 41 binaries, 0 generals, 0 SOSs, and 0 indicators.
Reduced MIQP objective Q matrix has 3 nonzeros.
Presolve time = 0.00 sec. (0.08 ticks)
Probing time = 0.00 sec. (0.04 ticks)
Tried aggregator 1 time.
Reduced MIQP has 20 rows, 44 columns, and 85 nonzeros.
Reduced MIQP has 41 binaries, 1 generals, 0 SOSs, and 0 indicators.
Reduced MIQP objective Q matrix has 3 non

Dual values of MILP couldn't be parsed


BG00 CCGT
Version identifier: 22.1.2.0 | 2024-12-10 | f4cec290b
CPXPARAM_Read_DataCheck                          1
CPXPARAM_LPMethod                                4
CPXPARAM_Threads                                 4
CPXPARAM_Emphasis_Numerical                      1
CPXPARAM_Emphasis_MIP                            4
CPXPARAM_MIP_Tolerances_MIPGap                   0.01
CPXPARAM_Barrier_ConvergeTol                     0.01
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIQP Presolve eliminated 5 rows and 0 columns.
Reduced MIQP has 1 rows, 5 columns, and 5 nonzeros.
Reduced MIQP has 4 binaries, 0 generals, 0 SOSs, and 0 indicators.
Reduced MIQP objective Q matrix has 1 nonzeros.
Presolve time = 0.00 sec. (0.00 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Reduced MIQP has 1 rows, 5 columns, and 5 nonzeros.
Reduced MIQP has 4 binaries, 1 generals, 0 SOSs, and 0 indicators.
Reduced MIQP objective Q matrix has 1 nonzeros.
Pr

...

Dual values of MILP couldn't be parsed


SK00 oil
UK00 oil
Version identifier: 22.1.2.0 | 2024-12-10 | f4cec290b
CPXPARAM_Read_DataCheck                          1
CPXPARAM_LPMethod                                4
CPXPARAM_Threads                                 4
CPXPARAM_Emphasis_Numerical                      1
CPXPARAM_Emphasis_MIP                            4
CPXPARAM_MIP_Tolerances_MIPGap                   0.01
CPXPARAM_Barrier_ConvergeTol                     0.01
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIQP Presolve eliminated 3 rows and 0 columns.
Reduced MIQP has 4 rows, 6 columns, and 9 nonzeros.
Reduced MIQP has 3 binaries, 0 generals, 0 SOSs, and 0 indicators.
Reduced MIQP objective Q matrix has 3 nonzeros.
Presolve time = 0.00 sec. (0.01 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Reduced MIQP has 4 rows, 6 columns, and 9 nonzeros.
Reduced MIQP has 3 binaries, 3 generals, 0 SOSs, and 0 indicators.
Reduced MIQP objective Q matrix has 3 nonz

Dual values of MILP couldn't be parsed


UKNI oil
DE00 hydrogen
NL00 hydrogen
UK00 hydrogen
BA00 OCGT
BA00 CCGT
CH00 OCGT
GR03 OCGT
GR03 CCGT
LV00 OCGT
ME00 OCGT
ME00 CCGT
NOM1 OCGT
NOM1 CCGT
NON1 OCGT
NON1 CCGT
NOS1 OCGT
NOS1 CCGT
NOS2 OCGT
NOS3 OCGT
SE02 OCGT
SE02 CCGT
Large decomposition finished successfully


/tmp/ipykernel_2832932/836708387.py:16: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  matched_plants = pd.concat(matched_plants)
/tmp/ipykernel_2832932/1553093801.py:3: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_plants = pd.concat([detailed_plants, generic_plants])


In [21]:
age_matching = get_age_matching()

In [22]:
age_matching

from      to
nuclear  -          1900.0  2040.0
coal     old 1      1900.0  1980.0
         old 2      1980.0  2010.0
         new        2010.0  2040.0
lignite  old 1      1900.0  1980.0
         old 2      1980.0  2010.0
         new        2010.0  2040.0
OCGT     old 1      1900.0  1980.0
         old 2      1980.0  2010.0
CCGT     old 1      1900.0  1980.0
         old 2      1980.0  2000.0
         present 1  2000.0  2010.0
         present 2  2010.0  2020.0
         new        2020.0  2040.0
OCGT     new        2010.0  2040.0
oil      old 1      1900.0  1980.0
         old 2      1980.0  2010.0
         new        2010.0  2040.0
hydrogen CCGT       1900.0  2040.0
         OCGT       1900.0  2040.0
biomass  -          1900.0  2040.0
other    -          1900.0  2040.0

In [23]:
    idx = pd.IndexSlice
    
    all_plants["age_class"] = np.nan
    
    for plant in detailed_plants.index:
    
        datein = power_plants.loc[plant, "DateIn"]
        carrier = power_plants.loc[plant, "carrier"]
        
        if (carrier in age_matching.index.levels[0]) and not np.isnan(datein):
            all_plants.loc[plant, "age_class"] = age_matching[(age_matching["from"] <= datein)&(age_matching["to"] > datein)].loc[carrier].index[0]
        elif (carrier in age_matching.index.levels[0]) and ("old 2" in age_matching.loc[carrier].index):
            all_plants.loc[plant, "age_class"] = "old 2"    
    

/tmp/ipykernel_2832932/2538078244.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'old 1' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  all_plants.loc[plant, "age_class"] = age_matching[(age_matching["from"] <= datein)&(age_matching["to"] > datein)].loc[carrier].index[0]


In [24]:
all_plants[all_plants.age_class.isna()]

,bus,carrier,p_nom,efficiency,cluster,invest_status,entry,exit,age_class
1684,FI00,biomass,35.0,NaN,FI00 biomass exit 2036,existing,2028,2036,NaN
1689,FI00,biomass,85.0,NaN,FI00 biomass exit 2036,existing,2028,2036,NaN
3431,ITSI,biomass,18.0,NaN,ITSI biomass exit 2036,existing,2028,2036,NaN
3437,ITCA,biomass,32.0,NaN,ITCA biomass exit 2036,existing,2028,2030,NaN
4488,BE00,biomass,12.0,NaN,BE00 biomass exit 2036,existing,2028,2030,NaN
...,...,...,...,...,...,...,...,...,...
UKNI oil exit 2036 3,UKNI,oil,50.0,NaN,UKNI oil exit 2036,existing,2028,2036.0,NaN
UKNI oil exit 2036 4,UKNI,oil,50.0,NaN,UKNI oil exit 2036,existing,2028,2036.0,NaN
UKNI oil exit 2036 5,UKNI,oil,50.0,NaN,UKNI oil exit 2036,existing,2028,2036.0,NaN
UKNI oil exit 2036 6,UKNI,oil,50.0,NaN,UKNI oil exit 2036,existing,2028,2036.0,NaN


In [29]:
all_plants.loc[all_plants.carrier.isin(["biomass", "nuclear", "other"]), "age_class"] = "-"

In [ ]:
    all_plants.age_class = all_plants.age_class.fillna("new")
    all_plants.loc[all_plants.carrier.isin(["biomass", "nuclear", "other"]), "age_class"] = "-"
    all_plants.loc[all_plants.carrier =="hydrogen", "age_class"] = "CCGT"
    
    all_plants['invest_status'] = "existing"
    all_plants['p_nom_max'] = all_plants.p_nom
    all_plants['p_nom_min'] = 0.01
    
    all_plants[['start_up_cost', 'ramp_limit_up', 'ramp_limit_down', 'p_min_pu', 'min_up_time', 'min_down_time', 'var_om' ]] = np.nan
    
    start_up_cost = []
    ramp_limit_up = []
    ramp_limit_down = []
    p_min_pu = []
    min_up_time = []
    min_down_time = [] 
    var_om = [] 
    efficiency = []
    
    for plant in all_plants.index:
    
        carrier = all_plants.loc[plant, "carrier"]
        age_class = all_plants.loc[plant, "age_class"]
        
        start_up_cost.append(technology_parameters.loc[idx[carrier, age_class], "start_up_fix_cost"])
        ramp_limit_up.append(technology_parameters.loc[idx[carrier, age_class], "ramp_limit_up"])
        ramp_limit_down.append(technology_parameters.loc[idx[carrier, age_class], "ramp_limit_down"])
        p_min_pu.append(technology_parameters.loc[idx[carrier, age_class], "p_min_pu"])
        min_up_time.append(technology_parameters.loc[idx[carrier, age_class], "min_up_time"])
        min_down_time.append(technology_parameters.loc[idx[carrier, age_class], "min_down_time"])
        var_om.append(technology_parameters.loc[idx[carrier, age_class], "var_OM"])
        efficiency.append(technology_parameters.loc[idx[carrier, age_class], "efficiency"])
    
    all_plants.start_up_cost = start_up_cost
    
    """
    all_plants.ramp_limit_down_real = ramp_limit_down
    all_plants.ramp_limit_down = all_plants.ramp_limit_down_real.clip(upper=1.) 
    all_plants.ramp_limit_up_real = ramp_limit_up
    all_plants.ramp_limit_up = all_plants.ramp_limit_up_real.clip(upper=1.)
    """
    
    all_plants.ramp_limit_down = ramp_limit_down
    all_plants.ramp_limit_up = ramp_limit_up
    all_plants.p_min_pu = p_min_pu
    all_plants.min_up_time = min_up_time
    all_plants.min_down_time = min_down_time
    all_plants.var_om = var_om
    all_plants.efficiency = efficiency

    all_plants.p_nom = all_plants.p_nom.astype(float)
    all_plants.p_nom_max = all_plants.p_nom_max.astype(float)

In [25]:
all_plants = add_technical_properties_existing(all_plants, detailed_plants, technology_parameters, age_matching)

/tmp/ipykernel_2831865/1434117774.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'old 1' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  eraa_plants_detailed.loc[plant, "age_class"] = age_matching[(age_matching["from"] <= datein)&(age_matching["to"] > datein)].loc[carrier].index[0]


In [26]:
all_plants.query("carrier == 'biomass'")

,bus,carrier,p_nom,efficiency,cluster,invest_status,entry,exit,age_class,p_nom_max,p_nom_min,start_up_cost,ramp_limit_up,ramp_limit_down,p_min_pu,min_up_time,min_down_time,var_om
480,BE00,biomass,268.000,0.41,BE00 biomass exit 2036,existing,2028,2036,-,268.000,0.01,45.42,9.0,9.0,0.2,5.0,5.0,1.1
801,FI00,biomass,185.000,0.41,FI00 biomass exit 2036,existing,2028,2036,-,185.000,0.01,45.42,9.0,9.0,0.2,5.0,5.0,1.1
1022,DE00,biomass,47.369,0.41,DE00 biomass exit 2036,existing,2028,2036,-,47.369,0.01,45.42,9.0,9.0,0.2,5.0,5.0,1.1
1044,DE00,biomass,15.564,0.41,DE00 biomass exit 2036,existing,2028,2036,-,15.564,0.01,45.42,9.0,9.0,0.2,5.0,5.0,1.1
1049,DE00,biomass,28.000,0.41,DE00 biomass exit 2036,existing,2028,2036,-,28.000,0.01,45.42,9.0,9.0,0.2,5.0,5.0,1.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
UK00 biomass exit 2036 42,UK00,biomass,50.000,0.41,UK00 biomass exit 2036,existing,2028,2036.0,new,50.000,0.01,45.42,9.0,9.0,0.2,5.0,5.0,1.1
UK00 biomass exit 2036 43,UK00,biomass,50.000,0.41,UK00 biomass exit 2036,existing,2028,2036.0,new,50.000,0.01,45.42,9.0,9.0,0.2,5.0,5.0,1.1
UK00 biomass exit 2036 44,UK00,biomass,50.000,0.41,UK00 biomass exit 2036,existing,2028,2036.0,new,50.000,0.01,45.42,9.0,9.0,0.2,5.0,5.0,1.1
UK00 biomass exit 2036 45,UK00,biomass,34.100,0.41,UK00 biomass exit 2036,existing,2028,2036.0,new,34.100,0.01,45.42,9.0,9.0,0.2,5.0,5.0,1.1


In [25]:
small_aggregated = aggregate_small(all_plants)

In [26]:
all_plants = build_new_investments(all_plants, technology_parameters)

In [27]:
small_aggregated = build_new_investments(small_aggregated, technology_parameters)

In [30]:
hdf_file = "resources/capacity_tables/individual_plants.h5"

In [31]:
os.makedirs(os.path.dirname(hdf_file), exist_ok=True)

In [32]:
all_plants.to_hdf(hdf_file, key="detailed")
small_aggregated.to_hdf(hdf_file, key="small_aggregated")

/tmp/ipykernel_2803205/3003810118.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->axis1] [items->None]

  all_plants.to_hdf(hdf_file, key="detailed")
/tmp/ipykernel_2803205/3003810118.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block1_values] [items->Index(['bus', 'carrier', 'cluster', 'invest_status', 'entry', 'exit',
       'age_class'],
      dtype='object')]

  all_plants.to_hdf(hdf_file, key="detailed")
/tmp/ipykernel_2803205/3003810118.py:2: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->axis1] [items->None]

  small_aggregated.to_hdf(hdf_file, key="small_aggregated")
/tmp/ipykernel_2803205/3003810118.py:2: PerformanceWarning: 
your 